In [3]:
%matplotlib inline
import numpy as np

> **作者**:	Gaël Varoquaux

[数学优化](http://en.wikipedia.org/wiki/Mathematical_optimization)处理寻找一个函数的最小值（最大值或零）的问题。在这种情况下，这个函数被称为*成本函数*，或*目标函数*，或*能量*。

这里，我们感兴趣的是使用[scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html#scipy.optimize)来进行黑盒优化： 我们不依赖于我们优化的函数的算术表达式。注意这个表达式通常可以用于高效的、非黑盒优化。

> 先决条件
- Numpy, Scipy
- matplotlib

---

**也可以看一下: 参考**

数学优化是非常 ... 数学的。如果你需要性能，那么很有必要读一下这些书:
- [Convex Optimization](http://www.stanford.edu/~boyd/cvxbook/) Boyd and Vandenberghe (pdf版线上免费)。
- [Numerical Optimization](http://users.eecs.northwestern.edu/~nocedal/book/num-opt.html), Nocedal and Wright。 关于梯度下降方法的详细参考。
- [Practical Methods of Optimization](http://www.amazon.com/gp/product/0471494631/ref=ox_sc_act_title_1?ie=UTF8&smid=ATVPDKIKX0DER) Fletcher: 擅长挥手解释。

**章节内容**
- 了解你的问题
    - 凸优化 VS 非凸优化
    - 平滑问题和非平滑问题
    - 嘈杂VS精确的成本函数
    - 限制
- 不同最优化方法的回顾
    - 入门: 一维最优化
    - 基于梯度的方法
    - 牛顿和拟牛顿法
    - 较少梯度方法
    - 全局优化
- 使用scipy优化的操作指南
    - 选择一个方法
    - 让你的优化器更快
    - 计算梯度
    - 虚拟练习
- 特殊情境: 非线性最小二乘
    - 最小化向量函数的范数
    - 曲线拟合
- 有限制的最优化
    - 箱边界
    - 通用限制
    
## 2.7.1 了解你的问题

每个问题都是不相同。了解你的问题使你可以选择正确的工具。

---
**问题的维数**

优化问题的规模非常好的由问题的维数来决定，即，进行搜索的标量变量的数量。

---

### 2.7.1.1 凸优化 VS 非凸优化

**凸函数**:
- $f$ 在它的所有切线之上。
- 相应的, 对于两个点point A, B, f(C) 在线段[f(A), f(B])]之下, 如果 A < C < B

![](http://www.scipy-lectures.org/_images/plot_convex_1.png)

**非凸函数**

![](http://www.scipy-lectures.org/_images/plot_convex_2.png)

**最优化凸函数简单。最优化非凸函数可能非常困难。**

> **注意**: 可以证明对于一个凸函数局部最小值也是全局最小值。然后，从某种意义上说，最小值是惟一的。

### 2.7.1.2 平滑和非平滑问题

**平滑函数**:

梯度无处不在，是一个连续函数

![](http://www.scipy-lectures.org/_images/plot_smooth_1.png)

**非平滑函数**:

![](http://www.scipy-lectures.org/_images/plot_smooth_2.png)

**优化平滑函数更简单一些** (在黑盒最优化的前提是对的，此外[线性编程](http://en.wikipedia.org/wiki/Linear_programming)是一个非常高效处理分段线性函数的例子)。

### 2.7.1.3 嘈杂 VS 精确成本函数

有噪音 (blue) 和无噪音 (green) 函数

![](http://www.scipy-lectures.org/_images/plot_noisy_1.png)

**噪音梯度**

许多优化方法都依赖于目标函数的梯度。如果没有给出梯度函数，会从数值上计算他们，会产生误差。在这种情况下，即使目标函数没有噪音，基于梯度的最优化也可能是噪音最优化。

### 2.7.1.4 限制

基于限制的最优化

这里是:

$-1 < x_1 < 1$

$-1 < x_2 < 1$

![](http://www.scipy-lectures.org/_images/plot_constraints_1.png)

## 2.7.2 不同最优化方法的回顾

### 2.7.2.1 入门: 一维最优化

使用[scipy.optimize.brent()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brent.html#scipy.optimize.brent) 来最小化一维函数。它混合抛物线近似与区间策略。

**二元函数的Brent方法**: 在3次迭代后收敛, 因为，稍后二元近似精确了。

![](http://www.scipy-lectures.org/_images/plot_1d_optim_1.png)

![](http://www.scipy-lectures.org/_images/plot_1d_optim_2.png)

**非凸函数的Brent方法**: 注意最优化方法避免了局部最小值其实是因为运气。

![](http://www.scipy-lectures.org/_images/plot_1d_optim_3.png)

![](http://www.scipy-lectures.org/_images/plot_1d_optim_4.png)

In [4]:
from scipy import optimize
def f(x):
    return -np.exp(-(x - .7)**2)
x_min = optimize.brent(f)  # 实际上在9次迭代后收敛!
x_min 

0.6999999997839409

In [4]:
x_min - .7 

-2.160590595323697e-10

> **注意**: Brent方法也可以用于*限制区间最优化*使用[scipy.optimize.fminbound()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fminbound.html#scipy.optimize.fminbound)

> **注意**: 在scipy 0.11中, [scipy.optimize.minimize_scalar()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html#scipy.optimize.minimize_scalar) 给出了一个一维标量最优化的通用接口。

### 2.7.2.2 基于梯度的方法

#### 2.7.2.2.1 关于梯度下降的一些直觉

这里我们关注**直觉**，不是代码。代码在后面。

从根本上说，梯度下降在于在梯度方向上前进小步，即最陡峭梯度的方向。

**固定步数梯度下降**

**状况良好的二元函数。**

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_0.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_100.png)

**状况糟糕的二元函数。**

状况糟糕问题的梯度下降算法的核心问题是梯度并不会指向最低点。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_2.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_102.png)

我们可以看到非常各向异性 (状况糟糕) 函数非常难优化。

---

**带回家的信息**: 条件数和预条件化

如果你知道变量的自然刻度，预刻度他们以便他们的行为相似。这与[预条件化](https://en.wikipedia.org/wiki/Preconditioner)相关。

---

并且，很明显采用大步幅是有优势的。这在梯度下降代码中使用[直线搜索](https://en.wikipedia.org/wiki/Line_search)。

**适应步数梯度下降**


状况良好的二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_1.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_101.png)

状况糟糕的二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_3.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_103.png)

状况糟糕的非二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_4.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_104.png)

状况糟糕的极端非二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_5.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_105.png)

函数看起来越像二元函数 (椭圆半圆边框线), 最优化越简单。

#### 2.7.2.2.2 共轭梯度下降

上面的梯度下降算法是玩具不会被用于真实的问题。

正如从上面例子中看到的，简单梯度下降算法的一个问题是，它试着摇摆穿越峡谷，每次跟随梯度的方法，以便穿越峡谷。共轭梯度通过添加*摩擦力*项来解决这个问题: 每一步依赖于前两个值的梯度然后急转弯减少了。

**共轭梯度下降**

状况糟糕的非二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_6.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_106.png)

状况糟糕的极端非二元函数。

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_7.png)

![](http://www.scipy-lectures.org/_images/plot_gradient_descent_107.png)

在scipy中基于共轭梯度下降方法名称带有‘cg’。最小化函数的简单共轭梯度下降方法是[scipy.optimize.fmin_cg()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_cg.html#scipy.optimize.fmin_cg):


In [5]:
def f(x):   # The rosenbrock函数
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
optimize.fmin_cg(f, [2, 2])  

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 120
         Gradient evaluations: 30


array([ 0.99998968,  0.99997855])

这些方法需要函数的梯度。方法可以计算梯度，但是如果传递了梯度性能将更好:

In [6]:
def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
optimize.fmin_cg(f, [2, 2], fprime=fprime)  

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 30
         Gradient evaluations: 30


array([ 0.99999199,  0.99998336])

注意函数只会评估30次，相对的没有梯度是120次。

### 2.7.2.3 牛顿和拟牛顿法

#### 2.7.2.3.1 牛顿法: 使用Hessian (二阶微分)

[牛顿法](http://en.wikipedia.org/wiki/Newton%27s_method_in_optimization)使用局部二元近似来计算跳跃的方向。为了这个目的，他们依赖于函数的前两个导数*梯度*和[Hessian](http://en.wikipedia.org/wiki/Hessian_matrix)。

**状况糟糕的二元函数:**

Note that, as the quadratic approximation is exact, the Newton method is blazing fast